# Setup Ensemble Model using weighted average of Predictions
## Imports

In [1]:
import helpers.set_path

from src.data.load_data import get_val_loader, get_test_loader, classes
from src.data.format_submissions import format_submissions

from pytorch_lightning import Trainer, seed_everything
import torch

import pandas as pd
import numpy as np
import wandb
from sklearn.metrics import log_loss, f1_score
from typing import List
from pytorch_lightning.loggers import WandbLogger

# Models
from src.models import Resnet50Model
from src.models import EfficientnetV2Model
from src.models import DensenetModel
from src.models import ViTModel
from src.models import VGG19Model
from src.models import InceptionV3Model

# Ensemble
from src.models import EnsembleModel

/home/luca/miniconda3/envs/ccv1-gpu/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/luca/miniconda3/envs/ccv1-gpu/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_RKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/luca/miniconda3/envs/ccv1-gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-13 17:21:31.740678: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

## Load the models to be averaged

In [2]:
def load_models_avg(model_paths: List[str]):
    models = []
    for i in range(len(model_paths)):
        if model_paths[i].startswith("../models/Resnet"):
            model = Resnet50Model.load_from_checkpoint(model_paths[i])
            models.append(model)
        elif model_paths[i].startswith("../models/Efficient"):
            model = EfficientnetV2Model.load_from_checkpoint(model_paths[i])
            models.append(model)
        elif model_paths[i].startswith("../models/Dense"):
            model = DensenetModel.load_from_checkpoint(model_paths[i])
            models.append(model)
        elif model_paths[i].startswith("../models/ViT"):
            model = ViTModel.load_from_checkpoint(model_paths[i])
            models.append(model)
        elif model_paths[i].startswith("../models/VGG"):
            model = VGG19Model.load_from_checkpoint(model_paths[i])
            models.append(model)
        elif model_paths[i].startswith("../models/InceptionV3Model"):
            model = InceptionV3Model.load_from_checkpoint(model_paths[i])
            models.append(model)

    return models

In [3]:
def get_sweep_name(model_paths: List[str]):
    sweep_name = ""
    for path in model_paths:
        filename = path.split("/")[-1]
        name = filename.split("_")[0]
        name = name[:-5]
        sweep_name += name + "_"
    return sweep_name[:-1]

In [4]:
model_paths = ["../models/DensenetModel.ckpt",
               "../models/EfficientnetV2Model.ckpt",
               "../models/InceptionV3Model.ckpt"]

models = load_models_avg(model_paths)
sweep_name = get_sweep_name(model_paths)

/home/luca/miniconda3/envs/ccv1-gpu/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/luca/miniconda3/envs/ccv1-gpu/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Sweep

In [5]:
sweep_config = {
    "name": "ensemble_average",
    "method": "grid",
    "parameters": {
        "weights": {"values": [[0.1, 0.1, 0.8],
                               [0.1, 0.8, 0.1],
                               [0.8, 0.1, 0.1],

                               [0.2, 0.2, 0.6],
                               [0.2, 0.6, 0.2],
                               [0.6, 0.2, 0.2],

                               [0.3, 0.3, 0.4],
                               [0.3, 0.4, 0.3],
                               [0.4, 0.3, 0.3],

                               [0.33, 0.33, 0.34]
                               ],
                               },
                            },
}

In [6]:
sweep_id = wandb.sweep(sweep=sweep_config, project="ccv1", entity="safari_squad")

def train_sweep():
    seed_everything(42)
    torch.set_float32_matmul_precision('high')
    
    run = wandb.init(project="ccv1", entity="safari_squad", config=sweep_config, name=sweep_name)
    config = run.config
    
    val_dataloader = get_val_loader(64, 224, 0.05)
    test_dataloader = get_test_loader(64, 224, 0.05)
        
    ensemble_model = EnsembleModel(models, config.weights)

    trainer = Trainer(accelerator="gpu")
    
    avg_preds = trainer.predict(ensemble_model, val_dataloader)
    
    avg_preds_numpy = [batch[1].cpu().detach().numpy() for batch in avg_preds] 
    avg_preds_numpy = np.concatenate(avg_preds_numpy)
    
    true_labels = [batch[2].numpy() for batch in avg_preds]
    true_labels = np.concatenate(true_labels)

    log_loss_value = log_loss(true_labels, avg_preds_numpy)

    avg_preds_numpy_argmax = np.argmax(avg_preds_numpy, axis=1)
    true_labels_argmax = np.argmax(true_labels, axis=1)


    f1_weighted = f1_score(true_labels_argmax, avg_preds_numpy_argmax, average="weighted")
    f1_macro = f1_score(true_labels_argmax, avg_preds_numpy_argmax, average="macro")
    f1_micro = f1_score(true_labels_argmax, avg_preds_numpy_argmax, average="micro")

    wandb.log({"log_loss": log_loss_value,
                "f1_weighted": f1_weighted,
                "f1_macro": f1_macro,
                "f1_micro": f1_micro})

    test_preds = trainer.predict(ensemble_model, test_dataloader)
    test_preds_numpy = [batch[1].cpu().detach().numpy() for batch in test_preds]
    test_preds_numpy = np.concatenate(test_preds_numpy)

    test_ids = [list(batch[0]) for batch in test_dataloader]
    test_ids = np.concatenate(test_ids)

    test_preds_df = pd.DataFrame(test_preds_numpy, columns=classes)
    test_preds_df.insert(0, "id", test_ids)
    
    wandb.log({"submission": wandb.Table(dataframe=test_preds_df)})
    
    wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 8vjbgkku
Sweep URL: https://wandb.ai/safari_squad/ccv1/sweeps/8vjbgkku


In [7]:
wandb.agent(sweep_id, function=train_sweep)

wandb: Agent Starting Run: 58uzw799 with config:
wandb: 	weights: [0.1, 0.1, 0.8]
Global seed set to 42
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: studentluca (safari_squad). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 54/54 [00:10<00:00,  5.05it/s]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 70/70 [00:10<00:00,  6.83it/s]


...

Predicting DataLoader 0: 100%|██████████| 54/54 [00:07<00:00,  6.96it/s]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 70/70 [00:10<00:00,  6.87it/s]


f1_macro,▁
f1_micro,▁
f1_weighted,▁
log_loss,▁
f1_macro,0.62987
f1_micro,0.66824
f1_weighted,0.66883
log_loss,0.96746


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
